---
#### DataGenerators : training and validation
---

In [39]:
import os
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image, ImageShow

In [12]:
from staintools.utils.visual import read_image, show, show_colors
from staintools.utils.visual import build_stack, patch_grid
from staintools import ReinhardNormalizer, MacenkoNormalizer, VahadaneNormalizer
from staintools import standardize_brightness

In [11]:
from sklearn.feature_extraction.image import extract_patches_2d
import skimage.io as ski

In [10]:
from keras.models import load_model, save_model, h5py
from keras.preprocessing.image import ImageDataGenerator, save_img, load_img, image, img_to_array, array_to_img, DirectoryIterator
from keras import optimizers
from keras.callbacks import TensorBoard, ModelCheckpoint, ProgbarLogger, Progbar
from keras import backend

In [8]:
# augment the dataset
# data_gen = ImageDataGenerator(horizontal_flip = True,
#                               rotation_range = 90,
#                               validation_split = .20, 
#                               vertical_flip = True,
#                               fill_mode = 'constant'
#                              )

# without augmentation
data_gen = ImageDataGenerator(validation_split=0.2)


# creating a generator for the training data reduces the memory burder
traingen = data_gen.flow_from_directory(batch_size=1, 
                                        directory="/Users/saqibnizami/notebooks/DSI-US-4/Capstone/datasets/training_patches/", 
                                        target_size=(512,512),
                                        subset="training",
#                                         save_to_dir='/Users/saqibnizami/notebooks/DSI-US-4/Capstone/datasets/aug_training',
#                                         save_prefix='training',
#                                         save_format='png',
                                        seed=19
                                        )

# validation set generator using the subset defined in ImageDataGenerator
# uses the last 20% of the training data as a validation set
valgen = data_gen.flow_from_directory(batch_size=1, 
                                        directory="/Users/saqibnizami/notebooks/DSI-US-4/Capstone/datasets/training_patches/", 
                                        target_size=(512,512),
                                        subset="validation",
#                                         save_to_dir='/Users/saqibnizami/notebooks/DSI-US-4/Capstone/datasets/aug_training',
#                                         save_prefix='validation',
#                                         save_format='png',
                                        seed=19
                                        )

Found 6972 images belonging to 4 classes.
Found 1743 images belonging to 4 classes.


In [9]:
# load the model

model = load_model("model.h5")

/Users/saqibnizami/anaconda3/envs/dsi/lib/python3.6/site-packages/keras/engine/saving.py:270: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [8]:
# make dir for logs
# !mkdir tensorboard_logs

In [6]:
opt = optimizers.Adam(lr=.003)
batch_size = 16
loss = 'categorical_crossentropy'
metrics = ['acc']
callbacks = [
    TensorBoard(log_dir = 'tensorboard_logs', histogram_freq = 1, embeddings_freq = 1, 
                batch_size=32, write_graph=True, write_grads=False, write_images=True,
                embeddings_layer_names=None, embeddings_metadata=None, 
                embeddings_data=None),
]

In [10]:
model.compile(optimizer=opt, 
              loss=loss,
              metrics=metrics)

In [11]:
history = model.fit_generator(generator = traingen,
                              verbose = 2,
                              callbacks = callbacks, 
                              validation_data=valgen, 
                              epochs=5, 
                              use_multiprocessing=True)

Epoch 1/5


Process ForkPoolWorker-1:
Traceback (most recent call last):
  File "/Users/saqibnizami/anaconda3/envs/dsi/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/saqibnizami/anaconda3/envs/dsi/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/saqibnizami/anaconda3/envs/dsi/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/Users/saqibnizami/anaconda3/envs/dsi/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/Users/saqibnizami/anaconda3/envs/dsi/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/Users/saqibnizami/anaconda3/envs/dsi/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/Users/saqibnizami/anaconda3/envs/dsi/lib/python3.6/multiprocessing/connection.py", line 379,

KeyboardInterrupt: 

In [3]:
model_trained = load_model("model_trained.h5")

In [4]:
# without augmentation
data_gen = ImageDataGenerator()


# creating a generator for the training data reduces the memory burder
testgen = data_gen.flow_from_directory(batch_size=1, 
                                        directory="/Users/saqibnizami/notebooks/DSI-US-4/Capstone/datasets/Test_data_patches//", 
                                        target_size=(512,512),
#                                         subset="training",
#                                         save_to_dir='/Users/saqibnizami/notebooks/DSI-US-4/Capstone/datasets/aug_training',
#                                         save_prefix='training',
#                                         save_format='png',
                                        seed=19
                                        )


Found 0 images belonging to 0 classes.


In [5]:
# 
# set paths to  variables here
test     = "datasets/Test_data_patches/*.png"


# create collection of images
tests     = ski.imread_collection(test)

In [7]:
model_trained.predict(tests.data)

ValueError: Error when checking input: expected l0_input to have 4 dimensions, but got array with shape (1, 1)

In [18]:
model_trained.predict(tests[0])

ValueError: Error when checking input: expected l0_input to have 4 dimensions, but got array with shape (512, 512, 3)

In [19]:
test[0].shape

AttributeError: 'str' object has no attribute 'shape'

In [56]:
test = read_image('datasets/Test_data_patches/0_p0.png')

In [62]:
test.shape = (1,) + test.shape

In [63]:
test

array([[[[237, 227, 246],
         [225, 217, 242],
         [229, 217, 243],
         ...,
         [228, 197, 239],
         [229, 198, 240],
         [231, 202, 241]],

        [[219, 209, 239],
         [234, 224, 245],
         [220, 207, 239],
         ...,
         [226, 192, 238],
         [228, 197, 239],
         [229, 199, 240]],

        [[196, 190, 230],
         [217, 208, 238],
         [236, 232, 247],
         ...,
         [228, 197, 239],
         [228, 197, 239],
         [229, 198, 240]],

        ...,

        [[229, 200, 240],
         [225, 195, 238],
         [225, 195, 238],
         ...,
         [243, 228, 248],
         [244, 232, 249],
         [245, 232, 249]],

        [[228, 199, 239],
         [229, 200, 240],
         [229, 200, 240],
         ...,
         [244, 231, 249],
         [244, 231, 249],
         [244, 230, 248]],

        [[228, 199, 239],
         [229, 200, 240],
         [229, 200, 240],
         ...,
         [244, 231, 249],
        

In [52]:
model_trained.predict(test)

array([[0.2766443 , 0.25165915, 0.23530707, 0.23638962]], dtype=float32)

In [67]:
model_trained.input_shape

(None, 512, 512, 3)

In [ ]:
model_trained.

In [ ]:
from keras.utils.visualize_util import plot  
plot(model_trained, to_file='model.png')  

def save_model_to_file(model, filename):
#     model.build()
    fig1 = plt.figure()
    ax1 = fig1.add_subplot(111, aspect='equal')
    ax1.axis('off')
    plt.xlim(model.x, model.x + model.width)
    plt.ylim(model.y + model.height, model.y)

    for feature_map in model.feature_maps + model.layers:
        for obj in feature_map.objects:
            if isinstance(obj, Line):
                if obj.dasharray == 1:
                    linestyle = ":"
                elif obj.dasharray == 2:
                    linestyle = "--"
                else:
                    linestyle = "-"
                plt.plot([obj.x1, obj.x2], [obj.y1, obj.y2], color=[c / 255 for c in obj.color], lw=obj.width,
                         linestyle=linestyle)
            elif isinstance(obj, Text):
                ax1.text(obj.x, obj.y, obj.body, horizontalalignment="center", verticalalignment="bottom",
                         size=2 * obj.size / 3, color=[c / 255 for c in obj.color])

    plt.savefig(filename)

save_model_to_file(model_trained, "model.png")

plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

from keras.utils.vis_utils import plot_model

plot_model(model_trained, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

import os
import keras

from keras.preprocessing import image
from keras.applications.imagenet_utils import decode_predictions
from skimage.io import imread

# I'm dividing by 2 and adding 0.5 because of how this Inception represents images
plt.imshow(test.shape[1:])

preds = model_trained.predict(test[0])
for x in decode_predictions(preds)[0]:
    print(x)

